<img style="float:right" width="25%" src="EscudoColorTc.jpg">
<h1 style="display: inline-block; padding: 4mm; padding-left= 2em; background-color: navy; line-height: 1.3em; color: white; border-radius: 10px">

    Práctica 6: Recomendación basada en memoria
</h1>

## Gestión de la Información. Curso 2025-2026

## Alumnos:
* Belén Rodríguez Cánovas
* Fernando Arroyo Redondo
___

# 1. Instalación de librerías necesarias

In [1]:
# Se importa la librería numpy que se va a usar para los vectores que se van a usar
import numpy as np
# Se importa la librería pandas para importar los ficheros csv que se van a necesitar
import pandas as pd
# Se importa la librería pyplot para la creación de gráficos
import matplotlib.pyplot as plt
# Se importa la librería de scipy que permite la optimización de los cálculos
from scipy.optimize import minimize
# Se importa las librerías de sklearn que permiten hacer imágenes de clusters
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# 2. Importación de datos a un Dataframe

In [2]:
# Diccionario de datos de usuarios y valoraciones
# Se almacena usando un Dataframe de pandas transformado luego en diccionario
# El fichero se ha creado usando los ficheros proporcionados en el enlace del problema
# Posteriormente se ha unificado en un fichero dataset.xlxs con los campos siguientes:
# userId, movieId, rating

# 1 Carga en el dataframe
df = pd.read_excel('datasetb.xlsx')

In [3]:
# Control de carga
print (df[:4])
print(".....")
print(df[-4:])

   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
.....
        userId  movieId  rating
100832     610   168248     5.0
100833     610   168250     5.0
100834     610   168252     5.0
100835     610   170875     3.0


# 3. Transformación del dataframe en diccionario

In [4]:
# Se agrupa por userId y luego transforma cada grupo en un dicionario donde la clave es movieId 
# y los valores Title y rating
data = {
    user_id:dict(zip(user_data['movieId'],user_data['rating'].apply(float)))
    for user_id, user_data in df.groupby('userId')
}
    


In [5]:
# Valoracián de una película en particular usuario 1
data[1][1] # Valoración usuario A película 'Toy Story'

4.0

In [6]:
# Valoracián de todas las películas evaluadas,en particular usuario 1
print(data[1])

{1: 4.0, 3: 4.0, 6: 4.0, 47: 5.0, 50: 5.0, 70: 3.0, 101: 5.0, 110: 4.0, 151: 5.0, 157: 5.0, 163: 5.0, 216: 5.0, 223: 3.0, 231: 5.0, 235: 4.0, 260: 5.0, 296: 3.0, 316: 3.0, 333: 5.0, 349: 4.0, 356: 4.0, 362: 5.0, 367: 4.0, 423: 3.0, 441: 4.0, 457: 5.0, 480: 4.0, 500: 3.0, 527: 5.0, 543: 4.0, 552: 4.0, 553: 5.0, 590: 4.0, 592: 4.0, 593: 4.0, 596: 5.0, 608: 5.0, 648: 3.0, 661: 5.0, 673: 3.0, 733: 4.0, 736: 3.0, 780: 3.0, 804: 4.0, 919: 5.0, 923: 5.0, 940: 5.0, 943: 4.0, 954: 5.0, 1009: 3.0, 1023: 5.0, 1024: 5.0, 1025: 5.0, 1029: 5.0, 1030: 3.0, 1031: 5.0, 1032: 5.0, 1042: 4.0, 1049: 5.0, 1060: 4.0, 1073: 5.0, 1080: 5.0, 1089: 5.0, 1090: 4.0, 1092: 5.0, 1097: 5.0, 1127: 4.0, 1136: 5.0, 1196: 5.0, 1197: 5.0, 1198: 5.0, 1206: 5.0, 1208: 4.0, 1210: 5.0, 1213: 5.0, 1214: 4.0, 1219: 2.0, 1220: 5.0, 1222: 5.0, 1224: 5.0, 1226: 5.0, 1240: 5.0, 1256: 5.0, 1258: 3.0, 1265: 4.0, 1270: 5.0, 1275: 5.0, 1278: 5.0, 1282: 5.0, 1291: 5.0, 1298: 5.0, 1348: 4.0, 1377: 3.0, 1396: 3.0, 1408: 3.0, 1445: 3.0, 1

# 4. Función para pasar del diccionario a las matrices Y(ratings), R(películas vistas)

In [7]:
def dic_ratings2arrays(dic):
    # Usuarios
    k1=np.unique(list(dic.keys())) #array usuarios
    
    # Películas
    k2=[]
    for i in k1:
        k2=k2 + list(dic[i].keys())

    k2=np.unique(k2) # array de películas

    # Valoraciones
    Y= np.zeros ((len(k1),len(k2))) #array de ratins creado  a cero
    # Relleno de valores de Y, recorriendo los dos vectores de usuarios y películas
    for i in k1:
        for j in k2:
            if j in dic[i].keys():
                Y[k1.tolist().index(i),k2.tolist().index(j)]= dic[i][j]
    return (Y,(Y>0)*1,k1,k2) #La matriz R es (Y>0)*1

# 5. Añadir dos usuarios

In [8]:
# En el paso 2, cuando se ha comprobado la carga, el último usuario era el 610, se añaden el 611 y el 612
data[611] = {1:4.7,552:4.3, 1023:4.5, 1270:4.0,2115:4.0, 30793:4.5}
data[612] ={1:3.5,552:3.2, 1090:4.5, 1092:4.6, 1208:4.0,2115:4.0, 20571:4.8, 99114:4.5, 912:4.0, 1079:3.9}

In [9]:
# Comprobar datos añadidos
print(data[611])
print(data[612])

{1: 4.7, 552: 4.3, 1023: 4.5, 1270: 4.0, 2115: 4.0, 30793: 4.5}
{1: 3.5, 552: 3.2, 1090: 4.5, 1092: 4.6, 1208: 4.0, 2115: 4.0, 20571: 4.8, 99114: 4.5, 912: 4.0, 1079: 3.9}


# 6. Creación de matrices Y y R

In [10]:
(Y,R,users,movies)=dic_ratings2arrays(data)

In [11]:
# Visualizacián de datos para ver algunos resultados, para ver que ha funcionado

print ('Matiz de valoraciones Y\n',Y[0]) # primer usuario de la matriz
print ('Matriz R\n',R[0]) # primera película de la matriz
print ('Usuarios\n',users[:10]) # 10 primeros usurios del conjunto de datos
print ('Películas\n',movies[:10])  # 10 primeras películas del conjunto de datos

Matiz de valoraciones Y
 [4. 0. 4. ... 0. 0. 0.]
Matriz R
 [1 0 1 ... 0 0 0]
Usuarios
 [ 1  2  3  4  5  6  7  8  9 10]
Películas
 [ 1  2  3  4  5  6  7  8  9 10]


# 7 Recomendar películas a nuevos usuarios

# Comentario sobre las características (n_features)

El fichero de movielens  tiene hasta 5 características para clasificar las películas.....

Y no tiene características para los usuarios.

In [12]:
def matrix_factorization(Y, R, n_features, lambda_reg=0.1, max_iterations=100, learning_rate=0.001):
    """
    Implementa una matriz de factorización para un sistema de recommendación
    
    Parámetros:
    -----------
    Y : Array de tipo numpy con la matriz de evaluaciones
        Y[i,j] es la evaluación de la película j hecha por el usuario i (ver punto 6)
    R : Array de tipo numpy con la matriz de películas vistas
        R[i,j] = 1 si i ha evaluado la película j, ó 0 en otro caso
    n_features : int
        Número de posibles características
    lambda_reg : float
        Parámetro de regularización
    max_iterations : int
        Máximo número de iteraciones para la optimización
    learning_rate : float
        porcentaje de aprindizaje para el gradiente
        
    Returns:
    --------
    X : Array de tipo numpy
        Matriz de características de los usuarios
    Theta : Array de tipo numpy
        Matriz de características de las películas
    """
    # Obtener dimensiones
    n_users, n_movies = Y.shape
    
    # Iniciar la matriz de características de forma aleatoria
    X = np.random.randn(n_users, n_features)  # Características de los usuarios
    Theta = np.random.randn(n_movies, n_features)  # Características de las películas

    # Definir la función de costes
    def cost_function(params):
        # Reestructurar los parámetro en X y Theta
        X_flat = params[:n_users * n_features].reshape(n_users, n_features)
        Theta_flat = params[n_users * n_features:].reshape(n_movies, n_features)
        
        # Calcular estimaciones
        predictions = np.dot(X_flat, Theta_flat.T)
        
        # Calcular el error (solamente se calcula para las películas evaluadas)
        error = np.sum(((predictions - Y) * R) ** 2) / 2
        
        # Añadir la regularización con lambda
        reg_X = (lambda_reg / 2) * np.sum(X_flat ** 2)
        reg_Theta = (lambda_reg / 2) * np.sum(Theta_flat ** 2)
        
        return error + reg_X + reg_Theta

    # Definir la función de gradiente
    def gradient(params):
        # Reestructurar los parámetro en X y Theta
        X_flat = params[:n_users * n_features].reshape(n_users, n_features)
        Theta_flat = params[n_users * n_features:].reshape(n_movies, n_features)
        
        # Calcular estimaciones
        predictions = np.dot(X_flat, Theta_flat.T)
        
        # Calcular errores
        errors = (predictions - Y) * R
        
        # Calculate gradientes
        X_grad = np.dot(errors, Theta_flat) + lambda_reg * X_flat
        Theta_grad = np.dot(errors.T, X_flat) + lambda_reg * Theta_flat
        
        # Aplanar los gradientes para facilitar su uso
        return np.concatenate([X_grad.flatten(), Theta_grad.flatten()])
    
    # Aplanar  X y Theta para su optimización
    initial_params = np.concatenate([X.flatten(), Theta.flatten()])
    
    # Mensaje de lanzamiento del entrenamiento con la optimización de scipy
    print("Entrenando el modelo basado en matriz de factorización...")
   
    result = minimize(
        fun=cost_function,
        x0=initial_params,
        method='L-BFGS-B',
        jac=gradient,
        options={'maxiter': max_iterations, 'disp': True}
    )
    
    # Reestructurar los parámetro optimizado hacia X y Theta
    optimized_params = result.x
    X = optimized_params[:n_users * n_features].reshape(n_users, n_features)
    Theta = optimized_params[n_users * n_features:].reshape(n_movies, n_features)
    
    print("Modelo de entrenamiento completado!")
    return X, Theta

In [24]:
# Datos para el entrenamiento
n_features = 2  # Número de características
lambda_reg = 0.1  # Parómetro de regularización

# Lanzar entrenamiento
user_features, movie_features = matrix_factorization(Y, R, n_features=n_features, lambda_reg=lambda_reg)


Entrenando el modelo basado en matriz de factorización...


C:\Users\35266\AppData\Local\Temp\ipykernel_30696\2699836135.py:77: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  result = minimize(


Modelo de entrenamiento completado!


In [25]:
# Función para obtener recomendaciones par un usuario específico
def get_recommendations(user_id, user_features, movie_features, movie_titles, n=10, already_seen=None):
    """
    Busca las N películas recomendadas a un usuario según las evaluaciones propias y ajenas
    
    Parameters:
    -----------
    user_id : int
        Indice que identifica al usuario en la matriz de características de usuarios
    user_features : numpy array
        Matriz de características de los usuarios
    movie_features : numpy array
        Matriz de características de las películas
    movie_titles : list or Series
        Lista de títulos de las películas con sus indicadores correspondientes
    n : int
        Número de recommendaciones a mostrar
    already_seen : numpy array
        Array con las películas vistas por el usuario
        
    Returns:
    --------
    lista de tuplas
        (movie_id, predicted_rating, movie_title)
    """
    # Obterner el vector de características de los usuario
    user_vector = user_features[user_id]
    
    # Calcular la predicción de evaluaciones de todas las películas
    predicted_ratings = np.dot(user_vector, movie_features.T)
    
    # Eliminar las películas ya vistas
    if already_seen is not None:
        predicted_ratings[already_seen == 1] = -np.inf
    
    # Obtener los identificadores de las N películas
    top_movie_indices = np.argsort(predicted_ratings)[::-1][:n]
    
    # Crear la lista de recommendatciones
    recommendations = [(idx, predicted_ratings[idx], movie_titles[idx]) 
                       for idx in top_movie_indices]
    
    return recommendations


In [26]:
# 1 Carga en el dataframe de títulos y películas
movies_df = pd.read_csv('peliculas.csv', sep=';')

In [27]:
# Control de carga
print (movies_df[:4])
print(".....")
print(movies_df[-4:])

   movieId                        Title
0        1             Toy Story (1995)
1        3      Grumpier Old Men (1995)
2        6                  Heat (1995)
3       47  Seven (a.k.a. Se7en) (1995)
.....
      movieId                             Title
9715   160527  Sympathy for the Underdog (1971)
9716   160836                     Hazard (2005)
9717   163937                Blair Witch (2016)
9718   163981                         31 (2016)


In [28]:
# Obtener recommendations para los usuarios 610 y 611
# Los títulos se obtienen del dataframe movies_df creado anteriormente
for user_id in [610, 611]:
    print(f"\n10 mejores recommendaciones para el Usuario {user_id}:")
    # Get movies this user has already seen
    already_seen = R[user_id]
    
    recommendations = get_recommendations(
        user_id, 
        user_features, 
        movie_features, 
        movie_titles=movies_df['Title'],  
        already_seen=already_seen
    )
    
    # Mostrar recomendaciones
    for i, (movie_id, pred_rating, title) in enumerate(recommendations, 1):
        print(f"{i}. {title} (Evaluación estimada: {pred_rating:.2f})")


10 mejores recommendaciones para el Usuario 610:
1. Flower of My Secret, The (La flor de mi secreto) (1995) (Evaluación estimada: 7.73)
2. Red Heat (1988) (Evaluación estimada: 7.56)
3. Delta Force 2 (Delta Force 2: The Colombian Connection) (1990) (Evaluación estimada: 7.22)
4. Grand Hotel (1932) (Evaluación estimada: 7.08)
5. Dragon Ball Z: Bio-Broly (Doragon bôru Z 11: Sûpâ senshi gekiha! Katsu no wa ore da) (1994) (Evaluación estimada: 6.96)
6. 1-900 (06) (1994) (Evaluación estimada: 6.82)
7. Things You Can Tell Just by Looking at Her (2000) (Evaluación estimada: 6.66)
8. V. I. Warshawski (1991) (Evaluación estimada: 6.61)
9. Wisdom (1986) (Evaluación estimada: 6.57)
10. Amateur (1994) (Evaluación estimada: 6.56)

10 mejores recommendaciones para el Usuario 611:
1. Flower of My Secret, The (La flor de mi secreto) (1995) (Evaluación estimada: 6.77)
2. Red Heat (1988) (Evaluación estimada: 6.63)
3. Delta Force 2 (Delta Force 2: The Colombian Connection) (1990) (Evaluación estimada: 

# 7. Función para calcular la correlación de Pearson

In [ ]:
# Correlacióon de Pearson (coincidencia en el espacio de valoraciones)
def pearson_similarity(Y,R,a,b):
    # Funciona con vectores fila
    common_ratings = (R[a,:]>0)&(R[b,:]>0) # Coincidencia en el espacio de valoraciones
    if common_ratings.any():
        # el promedio es sobre todas las valoraciones
        # Se consigue la media donde la matriz R tenga valores estrictamente positivos, 
        #lo que da una media sin tener en cuenta los ceros ver ejemplo en siguiente casilla
        a_average = Y[a,R[a,:]>0].mean() 
        b_average = Y[b,R[b,:]>0].mean()
        pcorr = np.dot(Y[a,common_ratings] - a_average,Y[b,common_ratings] - b_average)/(
                np.sqrt(np.dot(Y[a,common_ratings] - a_average,Y[a,common_ratings] - a_average))*
                np.sqrt(np.dot(Y[b,common_ratings] - b_average,Y[b,common_ratings] - b_average)))
        if np.isnan(pcorr): # En el caso de que se presente una indeterminación
            return 0
        else:
            return pcorr
    else: # En el caso de no haber una coincidencia en el espacio de valoraciones
        return 0

In [ ]:
print(Y[609,R[0,:]>0].mean()) #Media de valoración del usuario 0

In [ ]:
# Cálculo de la similitud de dos usuarios usuario 0 y usuario 2
for (u1,u2) in [(0,2)]:
    print('Similitud entre ',users[u1], 'y ',users[u2])
    print('Coeficiente de Pearson: %0.4f'% pearson_similarity(Y,R,u1,u2))

In [ ]:
# Cálculo de la similitud de todos los usuarios, si fuera necesario calcularla, el tamaño actual es muy grande
for u1 in range (Y.shape[0]):
    for u2 in range (u1 +1,Y.shape[0]):
        print('Similitud entre ( ',users[u1], ', ',users[u2], '): %0.4f'% pearson_similarity(Y,R,u1,u2))

# 8. Cálculo de la matriz de similitud

In [ ]:
#Cálculo de la matriz de similitud
def user_similarity_matrix(Y,R):
    matrix = np.zeros((Y.shape[0],Y.shape[0]))
    for u1 in range (Y.shape[0]):
        for u2 in range (u1,Y.shape[0]):
            matrix[u1,u2] = pearson_similarity(Y,R,u1,u2)
            if u1!=u2:
                matrix[u2,u1]=matrix[u1,u2]
    return matrix

# Mostrar la matriz. Si fuera necesario, el tamaño es muy grande
print ('Matriz de similitud\n',user_similarity_matrix(Y,R))

# 9. Encontrar usuarios que evaluaron Toys Story

In [ ]:
item_i=0 # el item 0 corresponde a Toys Story
k=np.where(R[:,item_i]>0)[0]
print (k)

In [ ]:
# Calcular la similitud de los k usuarios al usuario 610
user_sim = user_similarity_matrix(Y,R)
user_u=610 # El primero que se ha añadido
k_sim = user_sim[user_u,k]


In [ ]:
print(k_sim)

In [ ]:
# Unir índices de usuarios y similitudes: da una lista de tuplas
# primera posicián de la tupla con la similitud del usuario y de qué usuario se trata en segunda posisión
k_zip = list(zip(k_sim,k))
print ('Usuarios que vieron Toys Story: \n',k_zip)

In [ ]:
# Si se quiere eliminar los índices de usuarios con similitud negativa que distorsionan el coeficiente
k_zip=[k_zip[i] for i in range (len(k_zip)) if k_zip[i][0]> 0]
print ('Usuarios con similitud positiva que vieron Toys Story\n',k_zip)

In [ ]:
# Media de las valoraciones del usuario 610 (sobre las películas que si haya visto)
user_u_average=Y[user_u,R[user_u,:]>0].mean()
print(user_u_average)

In [ ]:
# Suma de los valores absolutos de similitud de vecinos próximos al usuario 610
k_sim=[k_zip[i][0] for i in range(len(k_zip))]
print(k_sim)
k_sim_suma=np.abs(np.array(k_sim)).sum()
print (k_sim_suma)

In [ ]:
# Estimación de la valoración de una película particular
rating_user_u_item_i=4
for user_v in [k_zip[i][1] for i in range (len(k_zip))]:
    rating_user_u_item_i += (Y[user_v,item_i] - Y[user_v,R[user_v,:]>0].mean())*user_sim[user_u,user_v]
rating_user_u_item_i = user_u_average + rating_user_u_item_i/k_sim_suma
print('Valoración estimada\n',rating_user_u_item_i)